In [1]:
!apt-get update
!apt-get install -y openmpi-bin openmpi-common libopenmpi-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,605 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,848 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,244 kB]
Get:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:13 http://archive.ubuntu.com/ub

In [32]:
# Write the Game of Life MPI code to a file
%%writefile game_of_life.c
#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>

// Function to count live neighbors for a cell
int count_neighbors(int *grid, int i, int j, int local_nx, int local_ny) {
    int count = 0;

    // Check all 8 neighboring cells
    for (int di = -1; di <= 1; di++) {
        for (int dj = -1; dj <= 1; dj++) {
            // Skip the cell itself
            if (di == 0 && dj == 0) continue;

            int ni = i + di;
            int nj = j + dj;

            // Check if neighbor is within bounds (including ghost cells)
            if (ni >= 0 && ni < local_nx + 2 && nj >= 0 && nj < local_ny + 2) {
                count += grid[ni * (local_ny + 2) + nj];
            }
        }
    }

    return count;
}

// Function to update the grid based on Conway's Game of Life rules
void update_grid(int *grid, int *new_grid, int local_nx, int local_ny) {
    for (int i = 1; i <= local_nx; i++) {
        for (int j = 1; j <= local_ny; j++) {
            int index = i * (local_ny + 2) + j;
            int live_neighbors = count_neighbors(grid, i, j, local_nx, local_ny);

            // Apply Conway's rules
            if (grid[index] == 1) {
                // Cell is alive
                if (live_neighbors < 2 || live_neighbors > 3) {
                    // Cell dies
                    new_grid[index] = 0;
                } else {
                    // Cell survives
                    new_grid[index] = 1;
                }
            } else {
                // Cell is dead
                if (live_neighbors == 3) {
                    // Cell becomes alive
                    new_grid[index] = 1;
                } else {
                    // Cell stays dead
                    new_grid[index] = 0;
                }
            }
        }
    }
}

// Function to print the local grid
void print_local_grid(int *grid, int local_nx, int local_ny, int rank, int generation) {
    printf("Rank %d - Generation %d:\n", rank, generation);
    for (int i = 1; i <= local_nx; i++) {
        for (int j = 1; j <= local_ny; j++) {
            printf("%d ", grid[i * (local_ny + 2) + j]);
        }
        printf("\n");
    }
    printf("\n");
}

// Function to randomly initialize the grid
void initialize_grid(int *grid, int local_nx, int local_ny, int seed) {
    srand(seed);

    // Set random values for the interior cells
    for (int i = 1; i <= local_nx; i++) {
        for (int j = 1; j <= local_ny; j++) {
            grid[i * (local_ny + 2) + j] = rand() % 2;
        }
    }

    // Initialize ghost cells to 0
    for (int i = 0; i < local_nx + 2; i++) {
        grid[i * (local_ny + 2)] = 0;
        grid[i * (local_ny + 2) + local_ny + 1] = 0;
    }

    for (int j = 0; j < local_ny + 2; j++) {
        grid[0 * (local_ny + 2) + j] = 0;
        grid[(local_nx + 1) * (local_ny + 2) + j] = 0;
    }
}

int main(int argc, char *argv[]) {
    int rank, size;
    int dims[2] = {0, 0};  // Let MPI decide the grid dimensions
    int periods[2] = {1, 1};  // Periodic boundaries
    int coords[2];
    int neighbors[4];  // North, East, South, West

    MPI_Comm cart_comm;
    MPI_Status status;

    // Initialize MPI
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Define the dimensions of the global grid
    int global_nx = 12;  // Global grid size in x
    int global_ny = 12;  // Global grid size in y
    int generations = 10;  // Number of generations to simulate

    // Check if we have command-line arguments for grid size and generations
    if (argc >= 3) {
        global_nx = atoi(argv[1]);
        global_ny = atoi(argv[2]);
    }
    if (argc >= 4) {
        generations = atoi(argv[3]);
    }

    // Create a 2D Cartesian topology
    MPI_Dims_create(size, 2, dims);
    MPI_Cart_create(MPI_COMM_WORLD, 2, dims, periods, 0, &cart_comm);
    MPI_Cart_coords(cart_comm, rank, 2, coords);

    // Get the ranks of neighboring processes
    MPI_Cart_shift(cart_comm, 0, 1, &neighbors[0], &neighbors[2]);  // North, South
    MPI_Cart_shift(cart_comm, 1, 1, &neighbors[3], &neighbors[1]);  // West, East

    // Calculate the local grid dimensions
    int local_nx = global_nx / dims[0];
    int local_ny = global_ny / dims[1];

    // Adjust for processes at the edge if necessary
    if (coords[0] == dims[0] - 1) {
        local_nx += global_nx % dims[0];
    }
    if (coords[1] == dims[1] - 1) {
        local_ny += global_ny % dims[1];
    }

    // Allocate memory for the local grid with ghost cells
    int *grid = (int *)malloc((local_nx + 2) * (local_ny + 2) * sizeof(int));
    int *new_grid = (int *)malloc((local_nx + 2) * (local_ny + 2) * sizeof(int));

    // Initialize the grid with random values
    initialize_grid(grid, local_nx, local_ny, rank + 1);

    // Arrays to store the data to be sent/received
    int *send_north = (int *)malloc(local_ny * sizeof(int));
    int *send_south = (int *)malloc(local_ny * sizeof(int));
    int *send_east = (int *)malloc(local_nx * sizeof(int));
    int *send_west = (int *)malloc(local_nx * sizeof(int));
    int *recv_north = (int *)malloc(local_ny * sizeof(int));
    int *recv_south = (int *)malloc(local_ny * sizeof(int));
    int *recv_east = (int *)malloc(local_nx * sizeof(int));
    int *recv_west = (int *)malloc(local_nx * sizeof(int));

    // Print initial configuration
    if (rank == 0) {
        printf("Process grid dimensions: %d × %d\n", dims[0], dims[1]);

        printf("Conway's Game of Life Parallel Implementation\n");
        printf("Global grid size: %d x %d\n", global_nx, global_ny);
        printf("Process grid: %d x %d\n", dims[0], dims[1]);
        printf("Generations: %d\n\n", generations);
    }
    printf("Rank %d handles grid of size %d × %d\n", rank, local_nx, local_ny);



    // Main simulation loop
    for (int gen = 0; gen < generations; gen++) {
        // Prepare the data to be sent
        // Copy the edge cells (excluding corners) to the send buffers
        for (int j = 0; j < local_ny; j++) {
            send_north[j] = grid[1 * (local_ny + 2) + j + 1];
            send_south[j] = grid[local_nx * (local_ny + 2) + j + 1];
        }

        for (int i = 0; i < local_nx; i++) {
            send_west[i] = grid[(i + 1) * (local_ny + 2) + 1];
            send_east[i] = grid[(i + 1) * (local_ny + 2) + local_ny];
        }

        // Exchange north-south ghost cells
        MPI_Sendrecv(send_north, local_ny, MPI_INT, neighbors[0], 0,
                     recv_south, local_ny, MPI_INT, neighbors[2], 0,
                     cart_comm, &status);

        MPI_Sendrecv(send_south, local_ny, MPI_INT, neighbors[2], 0,
                     recv_north, local_ny, MPI_INT, neighbors[0], 0,
                     cart_comm, &status);

        // Exchange east-west ghost cells
        MPI_Sendrecv(send_east, local_nx, MPI_INT, neighbors[1], 0,
                     recv_west, local_nx, MPI_INT, neighbors[3], 0,
                     cart_comm, &status);

        MPI_Sendrecv(send_west, local_nx, MPI_INT, neighbors[3], 0,
                     recv_east, local_nx, MPI_INT, neighbors[1], 0,
                     cart_comm, &status);

        // Copy the received data to the ghost cells
        for (int j = 0; j < local_ny; j++) {
            grid[0 * (local_ny + 2) + j + 1] = recv_north[j];
            grid[(local_nx + 1) * (local_ny + 2) + j + 1] = recv_south[j];
        }

        for (int i = 0; i < local_nx; i++) {
            grid[(i + 1) * (local_ny + 2) + 0] = recv_west[i];
            grid[(i + 1) * (local_ny + 2) + local_ny + 1] = recv_east[i];
        }

        // Exchange corner cells using MPI_Sendrecv with calculated neighbor ranks
        int nw_rank, ne_rank, sw_rank, se_rank;
        int nw_coord[2] = {(coords[0] - 1 + dims[0]) % dims[0], (coords[1] - 1 + dims[1]) % dims[1]};
        int ne_coord[2] = {(coords[0] - 1 + dims[0]) % dims[0], (coords[1] + 1) % dims[1]};
        int sw_coord[2] = {(coords[0] + 1) % dims[0], (coords[1] - 1 + dims[1]) % dims[1]};
        int se_coord[2] = {(coords[0] + 1) % dims[0], (coords[1] + 1) % dims[1]};

        MPI_Cart_rank(cart_comm, nw_coord, &nw_rank);
        MPI_Cart_rank(cart_comm, ne_coord, &ne_rank);
        MPI_Cart_rank(cart_comm, sw_coord, &sw_rank);
        MPI_Cart_rank(cart_comm, se_coord, &se_rank);

        // NW corner
        MPI_Sendrecv(&grid[1 * (local_ny + 2) + 1], 1, MPI_INT, nw_rank, 0,
                     &grid[0 * (local_ny + 2) + 0], 1, MPI_INT, nw_rank, 0,
                     cart_comm, &status);

        // NE corner
        MPI_Sendrecv(&grid[1 * (local_ny + 2) + local_ny], 1, MPI_INT, ne_rank, 0,
                     &grid[0 * (local_ny + 2) + local_ny + 1], 1, MPI_INT, ne_rank, 0,
                     cart_comm, &status);

        // SW corner
        MPI_Sendrecv(&grid[local_nx * (local_ny + 2) + 1], 1, MPI_INT, sw_rank, 0,
                     &grid[(local_nx + 1) * (local_ny + 2) + 0], 1, MPI_INT, sw_rank, 0,
                     cart_comm, &status);

        // SE corner
        MPI_Sendrecv(&grid[local_nx * (local_ny + 2) + local_ny], 1, MPI_INT, se_rank, 0,
                     &grid[(local_nx + 1) * (local_ny + 2) + local_ny + 1], 1, MPI_INT, se_rank, 0,
                     cart_comm, &status);

        // Update the grid based on Conway's rules
        update_grid(grid, new_grid, local_nx, local_ny);

        // Copy the new grid to the current grid
        memcpy(grid, new_grid, (local_nx + 2) * (local_ny + 2) * sizeof(int));

        // Print the grid at the specified generation
        if (gen == generations - 1) {
            MPI_Barrier(cart_comm);
            if (rank == 0) {
                printf("Final Generation %d\n", gen);
            }
            for (int r = 0; r < size; r++) {
                if (rank == r) {
                    print_local_grid(grid, local_nx, local_ny, rank, gen);
                }
                MPI_Barrier(cart_comm);
            }
        }
    }

    if (local_nx <= 0 || local_ny <= 0) {
    if (rank == 0) printf("Error: Some processes have zero-sized grids\n");
    MPI_Finalize();
    return 1;
}

    // Free allocated memory
    free(grid);
    free(new_grid);
    free(send_north);
    free(send_south);
    free(send_east);
    free(send_west);
    free(recv_north);
    free(recv_south);
    free(recv_east);
    free(recv_west);

    // Finalize MPI
    MPI_Finalize();

    return 0;
}

Overwriting game_of_life.c


In [33]:
!mpicc -O3 -o game_of_life game_of_life.c

In [23]:
!mpirun --oversubscribe --allow-run-as-root -np 4 ./game_of_life 8 8 10

Conway's Game of Life Parallel Implementation
Global grid size: 8 x 8
Process grid: 2 x 2
Generations: 10

Final Generation 9
Rank 0 - Generation 9:
0 0 0 0 
0 0 0 0 
0 0 0 0 
0 0 0 0 

Rank 1 - Generation 9:
0 1 0 0 
0 0 0 0 
0 0 0 0 
0 0 0 0 

Rank 2 - Generation 9:
0 0 0 0 
0 0 0 0 
0 0 0 0 
0 0 0 0 

Rank 3 - Generation 9:
0 1 1 0 
0 1 0 1 
1 0 0 1 
0 1 1 0 



In [34]:
print("\nRunning with 2 processes:")
!mpirun --allow-run-as-root --oversubscribe -np 2 ./game_of_life 8 8 10

# You can also try with a larger grid and more generations
print("\nRunning with a larger grid:")
!mpirun --allow-run-as-root --oversubscribe -np 4 ./game_of_life 16 16 20


Running with 2 processes:
Process grid dimensions: 2 × 1
Rank 1 handles grid of size 4 × 8
Conway's Game of Life Parallel Implementation
Global grid size: 8 x 8
Process grid: 2 x 1
Generations: 10

Rank 0 handles grid of size 4 × 8
Final Generation 9
Rank 0 - Generation 9:
0 0 0 0 0 0 0 0 
0 1 1 0 0 0 0 0 
1 1 0 1 1 0 0 0 
1 0 1 1 1 0 0 1 

Rank 1 - Generation 9:
1 0 0 0 1 0 1 1 
0 0 0 0 0 0 1 1 
1 0 0 0 0 0 0 1 
0 0 0 0 0 0 0 1 


Running with a larger grid:
Process grid dimensions: 2 × 2
Conway's Game of Life Parallel Implementation
Global grid size: 16 x 16
Process grid: 2 x 2
Generations: 20

Rank 0 handles grid of size 8 × 8
Rank 1 handles grid of size 8 × 8
Rank 3 handles grid of size 8 × 8
Rank 2 handles grid of size 8 × 8
Final Generation 19
Rank 0 - Generation 19:
0 1 0 0 0 1 0 1 
0 1 0 0 0 0 1 0 
0 0 1 0 0 1 1 1 
0 0 0 1 0 0 0 1 
0 0 0 0 0 1 0 0 
0 0 0 0 0 1 1 0 
0 1 1 0 0 1 0 1 
1 1 1 0 0 0 1 1 

Rank 1 - Generation 19:
0 0 1 1 0 0 0 0 
0 0 1 1 0 0 0 0 
0 1 0 0 0 0 0 0 
0 0

In [39]:
%%writefile game_of_life_fixed.c
#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>

// Function to count live neighbors for a cell
int count_neighbors(int *grid, int i, int j, int local_nx, int local_ny) {
    int count = 0;

    // Check all 8 neighboring cells
    for (int di = -1; di <= 1; di++) {
        for (int dj = -1; dj <= 1; dj++) {
            // Skip the cell itself
            if (di == 0 && dj == 0) continue;

            int ni = i + di;
            int nj = j + dj;

            // Check if neighbor is within bounds (including ghost cells)
            if (ni >= 0 && ni < local_nx + 2 && nj >= 0 && nj < local_ny + 2) {
                count += grid[ni * (local_ny + 2) + nj];
            }
        }
    }

    return count;
}

// Function to update the grid based on Conway's Game of Life rules
void update_grid(int *grid, int *new_grid, int local_nx, int local_ny) {
    for (int i = 1; i <= local_nx; i++) {
        for (int j = 1; j <= local_ny; j++) {
            int index = i * (local_ny + 2) + j;
            int live_neighbors = count_neighbors(grid, i, j, local_nx, local_ny);

            // Apply Conway's rules
            if (grid[index] == 1) {
                // Cell is alive
                if (live_neighbors < 2 || live_neighbors > 3) {
                    // Cell dies
                    new_grid[index] = 0;
                } else {
                    // Cell survives
                    new_grid[index] = 1;
                }
            } else {
                // Cell is dead
                if (live_neighbors == 3) {
                    // Cell becomes alive
                    new_grid[index] = 1;
                } else {
                    // Cell stays dead
                    new_grid[index] = 0;
                }
            }
        }
    }
}

// Function to print the local grid with better visualization
void print_local_grid(int *grid, int local_nx, int local_ny, int rank) {
    printf("Rank %d:\n", rank);
    for (int i = 1; i <= local_nx; i++) {
        for (int j = 1; j <= local_ny; j++) {
            printf("%s ", grid[i * (local_ny + 2) + j] ? "■" : "□");
        }
        printf("\n");
    }
    printf("\n");
}

// Function to gather and print the global grid (on rank 0)
void print_global_grid(int *local_grid, int local_nx, int local_ny, int global_nx, int global_ny,
                     int dims[2], int coords[2], MPI_Comm cart_comm, int rank, int size, int gen) {
    // Only rank 0 needs a buffer for the entire global grid
    int *global_grid = NULL;

    if (rank == 0) {
        global_grid = (int *)malloc(global_nx * global_ny * sizeof(int));
        memset(global_grid, 0, global_nx * global_ny * sizeof(int));
    }

    // Each process creates a buffer with just its local data (no ghost cells)
    int *local_data = (int *)malloc(local_nx * local_ny * sizeof(int));

    // Copy local grid data (excluding ghost cells) to the send buffer
    for (int i = 0; i < local_nx; i++) {
        for (int j = 0; j < local_ny; j++) {
            local_data[i * local_ny + j] = local_grid[(i + 1) * (local_ny + 2) + (j + 1)];
        }
    }

    // Information for gathering data to rank 0
    int *recvcounts = NULL;
    int *displs = NULL;

    if (rank == 0) {
        recvcounts = (int *)malloc(size * sizeof(int));
        displs = (int *)malloc(size * sizeof(int));
    }

    // Collect the size of each local grid
    int local_size = local_nx * local_ny;
    MPI_Gather(&local_size, 1, MPI_INT, recvcounts, 1, MPI_INT, 0, cart_comm);

    // Calculate displacements for gatherv
    if (rank == 0) {
        displs[0] = 0;
        for (int i = 1; i < size; i++) {
            displs[i] = displs[i-1] + recvcounts[i-1];
        }
    }

    // Allocate a temporary buffer for the gathered data
    int *gathered_data = NULL;
    if (rank == 0) {
        int total_size = 0;
        for (int i = 0; i < size; i++) {
            total_size += recvcounts[i];
        }
        gathered_data = (int *)malloc(total_size * sizeof(int));
    }

    // Gather all local data to rank 0
    MPI_Gatherv(local_data, local_nx * local_ny, MPI_INT, gathered_data,
               recvcounts, displs, MPI_INT, 0, cart_comm);

    // On rank 0, rearrange the gathered data into the global grid
    if (rank == 0) {
        // Clear global grid
        memset(global_grid, 0, global_nx * global_ny * sizeof(int));

        // Determine the base size of each local grid (without remainders)
        int base_nx = global_nx / dims[0];
        int base_ny = global_ny / dims[1];

        int pos = 0;
        for (int px = 0; px < dims[0]; px++) {
            for (int py = 0; py < dims[1]; py++) {
                // Calculate the actual size of this process's grid
                int proc_nx = base_nx + (px == dims[0]-1 ? global_nx % dims[0] : 0);
                int proc_ny = base_ny + (py == dims[1]-1 ? global_ny % dims[1] : 0);

                // Process rank in the Cartesian topology
                int proc_coords[2] = {px, py};
                int proc_rank;
                MPI_Cart_rank(cart_comm, proc_coords, &proc_rank);

                // Starting position in the global grid
                int start_x = px * base_nx;
                int start_y = py * base_ny;

                // Copy this process's data to the right position in the global grid
                for (int i = 0; i < proc_nx; i++) {
                    for (int j = 0; j < proc_ny; j++) {
                        int global_idx = (start_x + i) * global_ny + (start_y + j);
                        int local_idx = displs[proc_rank] + i * proc_ny + j;

                        if (local_idx < displs[proc_rank] + recvcounts[proc_rank]) {
                            global_grid[global_idx] = gathered_data[local_idx];
                        }
                    }
                }
            }
        }

        // Print the global grid with a nice header
        printf("\n======== Generation %d ========\n", gen);
        for (int i = 0; i < global_nx; i++) {
            for (int j = 0; j < global_ny; j++) {
                printf("%s ", global_grid[i * global_ny + j] ? "■" : "□");
            }
            printf("\n");
        }
        printf("\n");

        // Free resources
        free(gathered_data);
        free(recvcounts);
        free(displs);
        free(global_grid);
    }

    free(local_data);
}

// Function to randomly initialize the grid
void initialize_grid(int *grid, int local_nx, int local_ny, int seed) {
    srand(seed);

    // Set random values for the interior cells
    for (int i = 1; i <= local_nx; i++) {
        for (int j = 1; j <= local_ny; j++) {
            grid[i * (local_ny + 2) + j] = rand() % 2;
        }
    }

    // Initialize ghost cells to 0
    for (int i = 0; i < local_nx + 2; i++) {
        grid[i * (local_ny + 2)] = 0;
        grid[i * (local_ny + 2) + local_ny + 1] = 0;
    }

    for (int j = 0; j < local_ny + 2; j++) {
        grid[0 * (local_ny + 2) + j] = 0;
        grid[(local_nx + 1) * (local_ny + 2) + j] = 0;
    }
}

// Function to initialize with a glider pattern
void initialize_glider(int *grid, int local_nx, int local_ny, int coords[2], int dims[2], int global_nx, int global_ny) {
    // Clear the grid first
    for (int i = 0; i < (local_nx + 2) * (local_ny + 2); i++) {
        grid[i] = 0;
    }

    // Calculate the starting position in the global grid for this process
    int start_x = coords[0] * (global_nx / dims[0]);
    int start_y = coords[1] * (global_ny / dims[1]);

    // Glider pattern coordinates (in global grid)
    int glider[5][2] = {
        {1, 2},  // Top
        {2, 3},  // Right
        {3, 1},  // Bottom left
        {3, 2},  // Bottom middle
        {3, 3}   // Bottom right
    };

    // See if any part of the glider is in this process's domain
    for (int i = 0; i < 5; i++) {
        int global_x = glider[i][0];
        int global_y = glider[i][1];

        // Check if this cell is within this process's domain
        if (global_x >= start_x && global_x < start_x + local_nx &&
            global_y >= start_y && global_y < start_y + local_ny) {

            // Convert to local coordinates (add 1 for ghost cells)
            int local_x = global_x - start_x + 1;
            int local_y = global_y - start_y + 1;

            // Set the cell as alive
            grid[local_x * (local_ny + 2) + local_y] = 1;
        }
    }
}

// Function to derive corner ghost values from existing edge ghost values
// This avoids the need for explicit diagonal communication
void derive_corner_ghosts(int *grid, int local_nx, int local_ny) {
    // NW corner
    grid[0 * (local_ny + 2) + 0] = 0;

    // NE corner
    grid[0 * (local_ny + 2) + (local_ny + 1)] = 0;

    // SW corner
    grid[(local_nx + 1) * (local_ny + 2) + 0] = 0;

    // SE corner
    grid[(local_nx + 1) * (local_ny + 2) + (local_ny + 1)] = 0;
}

int main(int argc, char *argv[]) {
    int rank, size;
    int dims[2] = {0, 0};  // Let MPI decide the grid dimensions
    int periods[2] = {1, 1};  // Periodic boundaries
    int coords[2];
    int neighbors[4];  // North, East, South, West

    MPI_Comm cart_comm;
    MPI_Status status;

    // Initialize MPI
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Define the dimensions of the global grid
    int global_nx = 16;    // Global grid size in x
    int global_ny = 16;    // Global grid size in y
    int generations = 20;  // Number of generations to simulate
    int print_interval = 1; // Print every N generations
    int pattern = 0;       // 0 = random, 1 = glider

    // Check if we have command-line arguments for grid size and generations
    if (argc >= 3) {
        global_nx = atoi(argv[1]);
        global_ny = atoi(argv[2]);
    }
    if (argc >= 4) {
        generations = atoi(argv[3]);
    }
    if (argc >= 5) {
        print_interval = atoi(argv[4]);
    }
    if (argc >= 6) {
        pattern = atoi(argv[5]);
    }

    // Create a 2D Cartesian topology
    MPI_Dims_create(size, 2, dims);
    MPI_Cart_create(MPI_COMM_WORLD, 2, dims, periods, 0, &cart_comm);
    MPI_Cart_coords(cart_comm, rank, 2, coords);

    // Print process grid dimensions for debugging
    if (rank == 0) {
        printf("Process grid dimensions: %d × %d\n", dims[0], dims[1]);
    }

    // Get the ranks of neighboring processes
    MPI_Cart_shift(cart_comm, 0, 1, &neighbors[0], &neighbors[2]);  // North, South
    MPI_Cart_shift(cart_comm, 1, 1, &neighbors[3], &neighbors[1]);  // West, East

    // Calculate the local grid dimensions
    int local_nx = global_nx / dims[0];
    int local_ny = global_ny / dims[1];

    // Adjust for processes at the edge if necessary
    if (coords[0] == dims[0] - 1) {
        local_nx += global_nx % dims[0];
    }
    if (coords[1] == dims[1] - 1) {
        local_ny += global_ny % dims[1];
    }

    // Print local grid size for debugging
    printf("Rank %d handles grid of size %d × %d\n", rank, local_nx, local_ny);

    // Check for invalid grid sizes
    if (local_nx <= 0 || local_ny <= 0) {
        if (rank == 0) {
            printf("Error: Some processes have zero-sized grids. Try a different number of processes or larger grid.\n");
        }
        MPI_Finalize();
        return 1;
    }

    // Allocate memory for the local grid with ghost cells
    int *grid = (int *)malloc((local_nx + 2) * (local_ny + 2) * sizeof(int));
    int *new_grid = (int *)malloc((local_nx + 2) * (local_ny + 2) * sizeof(int));

    if (grid == NULL || new_grid == NULL) {
        printf("Error: Memory allocation failed on rank %d\n", rank);
        MPI_Abort(MPI_COMM_WORLD, 1);
    }

    // Initialize the grid (randomly or with a pattern)
    if (pattern == 0) {
        initialize_grid(grid, local_nx, local_ny, rank + 1);
    } else {
        initialize_glider(grid, local_nx, local_ny, coords, dims, global_nx, global_ny);
    }

    // Arrays to store the data to be sent/received
    int *send_north = (int *)malloc(local_ny * sizeof(int));
    int *send_south = (int *)malloc(local_ny * sizeof(int));
    int *send_east = (int *)malloc(local_nx * sizeof(int));
    int *send_west = (int *)malloc(local_nx * sizeof(int));
    int *recv_north = (int *)malloc(local_ny * sizeof(int));
    int *recv_south = (int *)malloc(local_ny * sizeof(int));
    int *recv_east = (int *)malloc(local_nx * sizeof(int));
    int *recv_west = (int *)malloc(local_nx * sizeof(int));

    if (send_north == NULL || send_south == NULL || send_east == NULL || send_west == NULL ||
        recv_north == NULL || recv_south == NULL || recv_east == NULL || recv_west == NULL) {
        printf("Error: Memory allocation failed for communication buffers on rank %d\n", rank);
        MPI_Abort(MPI_COMM_WORLD, 1);
    }

    // Print initial configuration
    if (rank == 0) {
        printf("Conway's Game of Life Parallel Implementation\n");
        printf("Global grid size: %d x %d\n", global_nx, global_ny);
        printf("Process grid: %d x %d\n", dims[0], dims[1]);
        printf("Generations: %d\n", generations);
        printf("Print interval: %d\n", print_interval);
        printf("Pattern: %s\n\n", pattern == 0 ? "Random" : "Glider");
    }

    // Print the initial global grid
    print_global_grid(grid, local_nx, local_ny, global_nx, global_ny,
                     dims, coords, cart_comm, rank, size, 0);

    // Main simulation loop
    for (int gen = 0; gen < generations; gen++) {
        // Prepare the data to be sent
        // Copy the edge cells (excluding corners) to the send buffers
        for (int j = 0; j < local_ny; j++) {
            send_north[j] = grid[1 * (local_ny + 2) + j + 1];
            send_south[j] = grid[local_nx * (local_ny + 2) + j + 1];
        }

        for (int i = 0; i < local_nx; i++) {
            send_west[i] = grid[(i + 1) * (local_ny + 2) + 1];
            send_east[i] = grid[(i + 1) * (local_ny + 2) + local_ny];
        }

        // Exchange north-south ghost cells
        MPI_Sendrecv(send_north, local_ny, MPI_INT, neighbors[0], 0,
                     recv_south, local_ny, MPI_INT, neighbors[2], 0,
                     cart_comm, &status);

        MPI_Sendrecv(send_south, local_ny, MPI_INT, neighbors[2], 0,
                     recv_north, local_ny, MPI_INT, neighbors[0], 0,
                     cart_comm, &status);

        // Exchange east-west ghost cells
        MPI_Sendrecv(send_east, local_nx, MPI_INT, neighbors[1], 0,
                     recv_west, local_nx, MPI_INT, neighbors[3], 0,
                     cart_comm, &status);

        MPI_Sendrecv(send_west, local_nx, MPI_INT, neighbors[3], 0,
                     recv_east, local_nx, MPI_INT, neighbors[1], 0,
                     cart_comm, &status);

        // Copy the received data to the ghost cells
        for (int j = 0; j < local_ny; j++) {
            grid[0 * (local_ny + 2) + j + 1] = recv_north[j];
            grid[(local_nx + 1) * (local_ny + 2) + j + 1] = recv_south[j];
        }

        for (int i = 0; i < local_nx; i++) {
            grid[(i + 1) * (local_ny + 2) + 0] = recv_west[i];
            grid[(i + 1) * (local_ny + 2) + local_ny + 1] = recv_east[i];
        }

        // Instead of exchanging corner cells, derive them from edge cells
        // This avoids the deadlock in corner exchanges
        derive_corner_ghosts(grid, local_nx, local_ny);

        // Update the grid based on Conway's rules
        update_grid(grid, new_grid, local_nx, local_ny);

        // Copy the new grid to the current grid
        memcpy(grid, new_grid, (local_nx + 2) * (local_ny + 2) * sizeof(int));

        // Print the global grid at specified intervals
        if ((gen + 1) % print_interval == 0 || gen == generations - 1) {
            print_global_grid(grid, local_nx, local_ny, global_nx, global_ny,
                             dims, coords, cart_comm, rank, size, gen + 1);
        }
    }

    // Free allocated memory
    free(grid);
    free(new_grid);
    free(send_north);
    free(send_south);
    free(send_east);
    free(send_west);
    free(recv_north);
    free(recv_south);
    free(recv_east);
    free(recv_west);

    // Finalize MPI
    MPI_Finalize();

    return 0;
}


Overwriting game_of_life_fixed.c


In [40]:
# Compile the fixed code
!mpicc -O3 -o game_of_life_fixed game_of_life_fixed.c

# Run with different numbers of processes to verify it works without deadlocks
print("\nRunning with 4 processes:")
!mpirun --allow-run-as-root --oversubscribe -np 4 ./game_of_life_fixed 16 16 10 5 1

print("\nRunning with 6 processes:")
!mpirun --allow-run-as-root --oversubscribe -np 6 ./game_of_life_fixed 16 16 10 5 1

print("\nRunning with 8 processes:")
!mpirun --allow-run-as-root --oversubscribe -np 8 ./game_of_life_fixed 16 16 10 5 1


Running with 4 processes:
Process grid dimensions: 2 × 2
Rank 0 handles grid of size 8 × 8
Conway's Game of Life Parallel Implementation
Rank 2 handles grid of size 8 × 8
Global grid size: 16 x 16
Process grid: 2 x 2
Generations: 10
Print interval: 5
Pattern: Glider

Rank 1 handles grid of size 8 × 8
Rank 3 handles grid of size 8 × 8

======== Generation 0 ========
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ 
□ □ ■ □ □ □ □ □ □ □ □ □ □ □ □ □ 
□ □ □ ■ □ □ □ □ □ □ □ □ □ □ □ □ 
□ ■ ■ ■ □ □ □ □ □ □ □ □ □ □ □ □ 
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ 
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ 
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ 
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ 
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ 
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ 
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ 
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ 
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ 
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ 
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ 
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ 


======== Generation 5 ========
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ 
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ 
□ □ 